In [19]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd 
from keras.layers import LSTM,Dropout,Dense
from keras.layers import Conv1D,MaxPooling1D,Flatten
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [20]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout
from keras.layers import *
from keras import backend as K

from sklearn.model_selection import KFold

import numpy as np
from numpy import linalg as LA
from itertools import product
import scipy
import random

import os
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm


In [21]:
import pandas as pd
df = pd.read_csv("/Users/rahuljauhari/Desktop/research-runoff/Data/merged_imd.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df = df.iloc[:, :157]

df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)
monthly_mean = df.resample('M').mean()
print(monthly_mean.shape)

df_actual = pd.read_excel("/Users/rahuljauhari/Desktop/research-runoff/Data/Calibrated and Validated.xlsx")
# select last column
observed_runnoff = df_actual['observed']
# observed_runnoff.head()
print(observed_runnoff.shape)

(468, 156)
(468,)


In [22]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # Nash-Sutcliffe efficiency
def nse(predictions, targets):
    return (1-(np.sum((predictions-targets)**2)/np.sum((targets-np.mean(targets))**2)))


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
scaler = MinMaxScaler(feature_range=(0,1))
x_scaled = scaler.fit_transform(monthly_mean)

In [25]:
x_train, x_test,y_train,y_test = train_test_split(x_scaled,observed_runnoff,test_size=0.2,shuffle=False)

In [26]:
x_train.shape

(374, 156)

FUZZY_BLOCK

In [27]:
n_neurons=100
n_feature= 5

In [28]:
# to get all permutaion
fRules = list(product([-1.0,0.0,1.0], repeat=n_feature)) 

# based on article just 100 of them are needed
out_fRules = random.sample(fRules, n_neurons)

fRules_sigma = K.transpose(out_fRules)

In [29]:
class fuzzy_inference_block(tf.keras.layers.Layer):
    def __init__(self, output_dim, i_fmap, mu, sigma):

        self.output_dim = output_dim
        self.index = i_fmap
        self.mu = mu
        self.sigma = sigma

        super(fuzzy_inference_block, self).__init__()

    def build(self, input_shape):
        self.mu_map = fRules_sigma * self.mu
        self.sigma_map = tf.ones((n_feature, self.output_dim)) * self.sigma
        

        super().build(input_shape)

    def call(self, inputs):
        fMap = inputs[:,n_feature*(self.index):n_feature*(self.index+1)]
        # create variables for processing
        aligned_x = K.repeat_elements(K.expand_dims(fMap, axis=-1), self.output_dim, -1)
        aligned_c = self.mu_map
        aligned_s = self.sigma_map


        # calculate output of each neuron (fuzzy rule)
        phi = K.exp(-K.sum(K.square(aligned_x - aligned_c) / (2 * K.square(aligned_s)),
                           axis=-2, keepdims=False))
        return phi


In [30]:
def fcnn(img_path='network_image.png', n_femap=4, stride=2, mu=3.0, sigma=1.2, dropout=True):
    # if stride is 3 => size of feature map will be 2x2
    # elif stride is 2 => size of feature map will be 3x3

    num_classes = 3

    inp = Input((x_train.shape[1],1))

    conv1 = Conv1D(3, (2), padding='valid', activation='tanh')(inp)
    conv1 = MaxPooling1D((2), strides=(1), padding='same')(conv1)

    conv2 = Conv1D(3, (2), padding='valid', activation='tanh')(conv1)
    conv2 = MaxPooling1D((2), strides=(1), padding='same')(conv2)
    
    conv3 = Conv1D(3, (2), padding='valid', activation='tanh')(conv2)
    conv3 = MaxPooling1D((2), strides=(1), padding='same')(conv3)

    conv4 = Conv1D(3, (2), padding='valid', activation='tanh')(conv3)
    conv4 = MaxPooling1D((2), strides=(1), padding='same')(conv4)

    conv5 = Conv1D(3, (2), padding='valid', activation='tanh')(conv4)
    conv5 = MaxPooling1D((2), strides=(1), padding='same')(conv5)

    conv6 = Conv1D(3, (2), padding='valid', activation='tanh')(conv5)
    conv6 = MaxPooling1D((2), strides=(1), padding='same')(conv6)

    conv7 = Conv1D(3, (2), padding='valid', activation='tanh')(conv6)
    conv7 = MaxPooling1D((2), strides=(1), padding='same')(conv7)

    conv8 = Conv1D(3, (2), padding='valid', activation='tanh')(conv7)
    conv8 = MaxPooling1D((2), strides=(1), padding='same')(conv8)

    conv9 = Conv1D(3, (2), padding='valid', activation='tanh')(conv8)
    conv9 = MaxPooling1D((2), strides=(1), padding='same')(conv9)

    fMaps = Flatten()(conv9)


    fuzzy_inference = []
    for i in tqdm(range(n_femap)):
        f_block = fuzzy_inference_block(output_dim=n_neurons, i_fmap=i, mu=mu, sigma=sigma)(fMaps)
        fuzzy_inference.append(f_block)
    merged = concatenate(fuzzy_inference, axis=1)

    out = Dense(3, activation='tanh')(merged)

    model = tf.keras.Model(inp, out)

    return model

In [31]:
model = fcnn(n_femap=50)

100%|██████████| 50/50 [00:00<00:00, 85.60it/s]


In [32]:
model.compile(optimizer="adam", loss='mse',metrics='accuracy')

In [33]:

model.fit(x_train, y_train,validation_split=0.1, batch_size=10, epochs=2,shuffle=False, use_multiprocessing=True)

Epoch 1/2
34/34 [==============================] - 12s 51ms/step - loss: 3029431.7500 - accuracy: 0.0000e+00 - val_loss: 2732183.7500 - val_accuracy: 0.0000e+00
Epoch 2/2
34/34 [==============================] - 0s 9ms/step - loss: 3028737.2500 - accuracy: 0.0000e+00 - val_loss: 2732107.2500 - val_accuracy: 0.0000e+00


In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
def func(name,x,y):
    inv = 0

    if name == 'StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)
        y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))
        x = x_scaled
        y = y_scaled
        inv = scaler

    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0, 1))
        x_scaled = scaler.fit_transform(x)
        y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))
        x = x_scaled
        y = y_scaled
        inv = scaler
        
    return x, y, inv

In [35]:
from sklearn.model_selection import train_test_split
x,y,inv = func('MinMaxScaler',monthly_mean,observed_runnoff)
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=False, test_size=0.2)

In [37]:
y_pred_train = model.predict(x_test)
y_pred_test = model.predict(x_test)
column_1 = y_pred_train[:, 0]
column_2 = y_pred_train[:, 1]
column_3 = y_pred_train[:, 2]

a = inv.inverse_transform(column_1.reshape(-1,1))
b = inv.inverse_transform(column_2.reshape(-1,1))
c = inv.inverse_transform(column_3.reshape(-1,1))
columns_to_save = np.column_stack((a,b,c))

# Define the file name
file_name = "fcnn test.csv"

# Save the columns to a CSV file
np.savetxt(file_name, columns_to_save, delimiter=',', header='Column 1,Column 2,Column 3', comments='')

print(f"The three columns have been saved to {file_name}.")

3/3 [==============================] - 0s 10ms/step
The three columns have been saved to fcnn test.csv.


In [ ]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)+K.epsilon()
    beta = np.mean(yp) / np.mean(yt)+K.epsilon()
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
# Normalized standard Error 
def nse1(yt, yp): 
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)

In [ ]:
yp1 = model.predict(x_train)

In [ ]:
yp1[0:10]

In [ ]:
yp2=model.predict(x_test)

In [ ]:
yp2[5]

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print(r2_score(y_train,yp1))

In [ ]:
print(r2_score(y_test,yp2))